# Chinese-CLIP

In [21]:
from PIL import Image
import requests
from transformers import ChineseCLIPProcessor, ChineseCLIPModel, ChineseCLIPConfig
from transformers import T5Tokenizer
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
import torch
import MLP

/data/mty/miniconda3/envs/py39/lib/python3.9/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/data/mty/miniconda3/envs/py39/lib/python3.9/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [24]:
# configuration = ChineseCLIPConfig(hidden_size = 1024, num_hidden_layers=22)
# model = ChineseCLIPModel(configuration)

chinese_clip_model = "/data/mty/UF-FGTG_code/train/cache/models/models--OFA-Sys--chinese-clip-vit-base-patch16/snapshots/36e679e65c2a2fead755ae21162091293ad37834"
model = ChineseCLIPModel.from_pretrained(chinese_clip_model)
# print("ChineseCLIP: ", model)
text_encoder = model.text_model
# print("ChineseCLIP: ", text_encoder)
processor = ChineseCLIPProcessor.from_pretrained("/data/mty/UF-FGTG_code/train/cache/models/models--OFA-Sys--chinese-clip-vit-base-patch16/snapshots/36e679e65c2a2fead755ae21162091293ad37834")
tokenizer = processor.tokenizer
# print(processor.model_max_length)
# print(processor)
print(processor.tokenizer)
# print(processor.tokenizer.max_model_input_sizes['bert-large-uncased'])
print(processor.tokenizer.max_model_input_sizes)
print(processor.tokenizer.model_max_length)

BertTokenizerFast(name_or_path='/data/mty/UF-FGTG_code/train/cache/models/models--OFA-Sys--chinese-clip-vit-base-patch16/snapshots/36e679e65c2a2fead755ae21162091293ad37834', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=F

In [27]:
prompt = "绵长蜿蜒的海滩，热带风情，明亮，简单"

input_ids_qwen = tokenizer(prompt, padding=True, return_tensors="pt").input_ids
print(input_ids_qwen.size())

# embeddings_CC = model.get_text_features(**input_CC)
output = text_encoder(input_ids_qwen)
print(output, output[0].size())

# print(embeddings_CC, embeddings_CC.size())


torch.Size([1, 20])
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.4692, -0.5100,  1.3219,  ..., -0.6875, -0.5799,  0.7358],
         [ 0.1718,  0.3449,  0.5172,  ..., -0.2099, -0.9013,  0.5558],
         [ 0.0459,  0.3314,  0.6434,  ..., -0.3581, -0.8340,  0.5171],
         ...,
         [-0.4529,  0.5153,  0.5547,  ..., -0.2163, -1.1226,  0.4805],
         [-0.4195,  0.4566,  0.5859,  ..., -0.2930, -1.1560,  0.5061],
         [-0.4692, -0.5100,  1.3219,  ..., -0.6875, -0.5799,  0.7358]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=None, hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None) torch.Size([1, 20, 768])


In [36]:
base_diffuser_model = "/data/mty/UF-FGTG_code/train/cache/models--stabilityai--stable-diffusion-2-1-base/snapshots/5ede9e4bf3e3fd1cb0ef2f7a3fff13ee514fdf06"
weight_dtype = torch.float32
torch.set_default_dtype(weight_dtype)
# device = "cuda:0"

pipe = StableDiffusionPipeline.from_pretrained(base_diffuser_model, torch_dtype=weight_dtype)
print(pipe.tokenizer.model_max_length)
print(pipe.tokenizer.max_model_input_sizes['openai/clip-vit-base-patch32'])

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
url = "https://clip-cn-beijing.oss-cn-beijing.aliyuncs.com/pokemon.jpeg"
image = Image.open(requests.get(url, stream=True).raw)
# Squirtle, Bulbasaur, Charmander, Pikachu in English
texts = ["杰尼龟", "妙蛙种子", "小火龙", "皮卡丘"]
# prompt = texts[3]
prompt = "绵长蜿蜒的海滩，热带风情，明亮，简单"

In [33]:
input_CC = processor(prompt, padding=True, return_tensors="pt")
input_ids_CC = processor(prompt, return_tensors="pt").input_ids
print(input_ids_CC.size())
input_ids_SD = pipe.tokenizer(prompt, return_tensors="pt").input_ids
print(input_ids_SD.size())    


# embeddings_CC = model.get_text_features(**input_CC)
embeddings_CC = text_encoder(input_ids_CC)["last_hidden_state"]
print(embeddings_CC, embeddings_CC.size())

embeddings_SD = pipe.text_encoder(input_ids_SD)["last_hidden_state"]
print(embeddings_SD.size())

torch.Size([1, 20])


NameError: name 'pipe' is not defined

# Qwen1.8B


In [31]:
CUDA_VISIBLE_DEVICES=0

from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model_path = "/data/share/models/Qwen1.5-1.8B"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at /data/share/models/Qwen1.5-1.8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
print(model)

Qwen2ForSequenceClassification(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048, padding_idx=151643)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qw

In [32]:
prompt = "绵长蜿蜒的海滩，热带风情，明亮，简单"

input_ids_qwen = tokenizer(prompt, padding=True, return_tensors="pt").input_ids
print(input_ids_qwen.size())

# embeddings_CC = model.get_text_features(**input_CC)
output = model(input_ids_qwen)
print(output, output.logits.size())

# print(embeddings_CC, embeddings_CC.size())


torch.Size([1, 13])
SequenceClassifierOutputWithPast(loss=None, logits=tensor([[ 2.0316, -5.0074, -0.7270,  ...,  8.7903, -1.2175,  1.4282]],
       grad_fn=<IndexBackward0>), past_key_values=((tensor([[[[-0.2311,  0.4273, -0.4108,  ..., -0.5478,  0.4174, -0.7064],
          [ 0.3504,  0.5560, -0.8452,  ..., -0.2257,  0.4423, -0.2694],
          [ 0.4751,  0.1811, -0.4068,  ..., -0.9261,  0.4448, -0.9573],
          ...,
          [ 0.0797,  0.2619, -0.3894,  ..., -0.5879,  0.4737, -0.8720],
          [-0.0394,  0.1008, -0.1894,  ..., -1.3103,  0.5756, -0.7676],
          [-0.1734, -0.3010, -0.3639,  ..., -0.4211,  0.4906, -0.5492]],

         [[-0.0409, -0.1928, -0.7332,  ...,  2.1198,  0.7853, -1.0449],
          [-0.6879,  0.3536, -0.3805,  ...,  2.5395,  1.6668, -1.4977],
          [-0.8619,  0.8144,  0.5397,  ...,  2.2697,  1.0385, -1.1954],
          ...,
          [ 0.4380,  0.8685, -0.4445,  ...,  2.1549,  0.7661, -1.0842],
          [ 0.4999,  0.8296,  0.0812,  ...,  3.4260,  

In [30]:
print(output.past_key_values)

None


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

# Initialize the Qwen model and tokenizer
model_path = "/home/mty/Qwen1.5-1.8B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
history = []
user_input = "夫妻肺片是什么？"
system_prompt = None
response, history = model.chat(tokenizer, user_input, history=history, system=system_prompt)
response

AttributeError: 'Qwen2ForCausalLM' object has no attribute 'chat'

In [9]:
prompt = "为什么我的爷爷奶奶没有邀请我参加他们的婚礼"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt")

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=128
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

'长辈们的婚礼可能有以下几个原因导致他们没有邀请你参加：\n\n1. 家庭传统和习俗：在某些文化或社会中，家庭的传统和习俗可能会对家族的仪式、庆祝活动以及新人的婚姻有着特定的要求和规定。例如，一些地区的传统婚礼必须在特定的时间、地点进行，或者在某个特定的节日或季节举行。如果你们的婚礼符合这些规定或风俗，那么可能是因为爷爷奶奶认为这样的安排更符合家族的传统。\n\n2. 财务问题：如果你的爷爷奶奶经济状况较好，且愿意为他们的孙子孙女提供更多的支持和参与度'

## LLAMA2-Chinese

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch
import torch.nn as nn

CUDA_VISIBLE_DEVICES=6

torch.cuda.empty_cache()

weight_dtype = torch.float32
torch.set_default_dtype(weight_dtype)
device = "cuda:0"

model_path = "/data/mty/UF-FGTG_code/train/cache/models/models--LinkSoul--Chinese-Llama-2-7b/snapshots/72efd71d7f89d9c46008b7a574faf90300ed9ba8"

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_path).to(weight_dtype).to(device)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# print(tokenizer.model_max_length) # 4096
# print("LLaMA2 Model: ", model)
# print("Streamer: ", streamer)

instruction = """[INST] <<SYS>>\n
            You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. 
            Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. 
            If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
            Please expand the following text in English so that it applies to the generated images. \n<</SYS>>\n
            \n{} [/INST]"""


prompt = instruction.format("绵长蜿蜒的海滩")
# prompt = instruction.format("一个漫画画像，描绘了一个赛博朋克的机械少女")

print(prompt)

input_LC = tokenizer(prompt, return_tensors='pt')
input_ids_LS = input_LC.input_ids.cuda()
# print(input_ids_LS.shape)

# generate_ids = model.generate(input_ids_LS, max_new_tokens=4096, streamer=streamer)
# print(generate_ids.shape)

embedding_LC = model(input_ids_LS)
# print(embedding_LC)
print(embedding_LC.logits.shape)

linear = nn.Linear(32000, 1024, dtype=weight_dtype).to(device)
embedding_LC = linear(embedding_LC.logits)
print(embedding_LC.shape)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[INST] <<SYS>>

            You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. 
            Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. 
            If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
            Please expand the following text in English so that it applies to the generated images. 
<</SYS>>

            
绵长蜿蜒的海滩 [/INST]
torch.Size([1, 179, 32000])
torch.Size([1, 179, 1024])


## Stable Diffusion


In [ ]:
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler

base_diffuser_model = "/data/mty/UF-FGTG_code/train/cache/models/models--stabilityai--stable-diffusion-2-1-base/snapshots/5ede9e4bf3e3fd1cb0ef2f7a3fff13ee514fdf06"
pipe = StableDiffusionPipeline.from_pretrained(base_diffuser_model, torch_dtype=torch.float16)
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)

In [ ]:
from math import sqrt
import torch
import torch.nn as nn

class CalculateAttention(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, Q, K, V, mask):
        print(torch.transpose(K, -1, -2).size())
        
        attention = torch.matmul(Q, torch.transpose(K, -1, -2))
        print("attention after matmul: ", attention.size())
        # use mask
        attention = attention.masked_fill_(mask, -1e9) # 将掩码为 True 的位置的注意力分数设置为一个非常小的值（-1e9）
        print("attention after masked_fill: ",attention.size())
        attention = torch.softmax(attention / sqrt(K.size(-1)), dim=-1)
        attention = torch.matmul(attention, V)
        return attention

class Multi_CrossAttention(nn.Module):
    """
    forward时，第一个参数用于计算query，第二个参数用于计算key和value
    """
    def __init__(self,hidden_size,all_head_size,head_num):
        super().__init__()
        self.hidden_size = hidden_size       # 输入维度
        self.all_head_size = all_head_size     # 输出维度
        self.num_heads = head_num          # 注意头的数量
        self.h_size = all_head_size // head_num # 每个头的维度

        assert all_head_size % head_num == 0

        # W_Q,W_K,W_V (hidden_size,all_head_size)
        self.linear_q = nn.Linear(hidden_size, all_head_size, bias=False)
        self.linear_k = nn.Linear(hidden_size, all_head_size, bias=False)
        self.linear_v = nn.Linear(hidden_size, all_head_size, bias=False)
        self.linear_output = nn.Linear(all_head_size, hidden_size)

        # normalization
        self.norm = sqrt(all_head_size)

    def print(self):
        print(self.hidden_size,self.all_head_size)
        print(self.linear_k,self.linear_q,self.linear_v)
    
    def forward(self,x,y,attention_mask):
        """
        cross-attention: x,y是两个模型的隐藏层，将x作为q的输入，y作为k和v的输入
        """
        batch_size = x.size(0)
        print("batch size: ", batch_size)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)

        # q_s: [batch_size, num_heads, seq_length, h_size]
        q_s = self.linear_q(x).view(batch_size, -1, self.num_heads, self.h_size).transpose(1,2)
        # print(q_s.shape)
        
        # k_s: [batch_size, num_heads, seq_length, h_size]
        k_s = self.linear_k(y).view(batch_size, -1, self.num_heads, self.h_size).transpose(1,2)
        
        # v_s: [batch_size, num_heads, seq_length, h_size]
        v_s = self.linear_v(y).view(batch_size, -1, self.num_heads, self.h_size).transpose(1,2)

        attention_mask = attention_mask.eq(0) #如果元素的值等于 0，那么 eq 函数将返回 True，否则返回 False。
        # print("attention_mask: ", attention_mask)

        attention = CalculateAttention()(q_s, k_s, v_s, attention_mask)
        # attention : [batch_size , seq_length , num_heads * h_size]
        attention = attention.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads * self.h_size)
        
        # output : [batch_size , seq_length , hidden_size]
        output = self.linear_output(attention)

        return output

In [ ]:
class Attention(nn.Module):
    def __init__(self, d_model):
        super(Attention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)
        # self.concat = nn.Linear(d_model, d_model)
        # self.n_heads = n_heads

    def forward(self, input_Q, input_K, input_V):
        """
        input_Q: [batch_size, len_q, d_model] len_q是作为query的句子的长度，比如enc_inputs（2,5,512）作为输入，那句子长度5就是len_q
        input_K: [batch_size, len_k, d_model]
        input_V: [batch_size, len_v(len_k), d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        """
        residual, batch_size = input_Q, input_Q.size(0)
        
        len = input_Q.size(1)
        d_q = input_Q.size(-1)
        d_k = input_K.size(-1)
        d_v = input_V.size(-1)

        # 1）linear projection [batch_size, seq_len, d_model] ->  [batch_size, n_heads, seq_len, d_k/d_v]
        Q = (
            self.W_Q(input_Q).view(batch_size, -1, n_heads, d_q).transpose(1, 2)
        )  # Q: [batch_size, n_heads, len_q, d_q]
        K = (
            self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1, 2)
        )  # K: [batch_size, n_heads, len_k, d_k]
        V = (
            self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1, 2)
        )  # V: [batch_size, n_heads, len_v(=len_k), d_v]

        return Q, K, V
        # # 2）计算注意力
        # # 自我复制n_heads次，为每个头准备一份mask
        # attn_mask = attn_mask.unsqueeze(1).repeat(
        #     1, n_heads, 1, 1
        # )  # attn_mask: [batch_size, n_heads, seq_len, seq_len]
        # context = ScaledDotProductionAttention()(
        #     Q, K, V, attn_mask
        # )  # context: [batch_size, n_heads, len_q, d_v]

        # # 3）concat部分
        # context = torch.cat(
        #     [context[:, i, :, :] for i in range(context.size(1))], dim=-1
        # )
        # output = self.concat(context)  # [batch_size, len_q, d_model]
                
        # # context = context.transpose(1, 2).reshape(batch_size, -1, d_model)
        # # output = self.linear(context)
        
        # return nn.LayerNorm(d_model).cuda()(
        #     output + residual
        # )  # output: [batch_size, len_q, d_model]

        """        
        最后的concat部分，网上的大部分实现都采用的是下面这种方式（也是哈佛NLP团队的写法）
        context = context.transpose(1, 2).reshape(batch_size, -1, d_model)
        output = self.linear(context)
        但是我认为这种方式拼回去会使原来的位置乱序，于是并未采用这种写法，两种写法最终的实验结果是相近的
        """

In [2]:
import torch
CUDA_VISIBLE_DEVICES=4

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)
text_embedding = embedding_LC.to(device)
# print(text_embedding.device)

batch_size = text_embedding.size(0)
seq_len = text_embedding.size(1)
embed_dim = text_embedding.size(2)

noise_embedding = torch.rand(1, seq_len, embed_dim)
noise_embedding = noise_embedding.to(device)
# print(noise_embedding.device)

n_heads = 16

# attention = Attention(embed_dim).to(device) # Create the instance of the class
# query, key, value = attention(text_embedding, noise_embedding, noise_embedding)
# print(query.shape, key.shape, value.shape)

# query = W_Q(text_embedding).view(batch_size, -1, n_heads, embed_dim).transpose(1, 2)

# attention_mask = torch.triu(torch.ones(1, seq_len, seq_len)) == 1
# print(attention_mask.shape)

# Cross Attention
multihead_attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=n_heads, dropout=0.1, batch_first=True).to(device)
attn_output, attn_output_weights = multihead_attn(query=text_embedding, key=noise_embedding, value=noise_embedding)

print(attn_output.shape, attn_output_weights.shape)

# layer = Multi_CrossAttention(1024, 1024, 16)

# cross_output = layer(random_noise, ramdon_embedding, attention_mask)

# print(cross_output.shape)

torch.Size([1, 179, 1024]) torch.Size([1, 179, 179])


In [ ]:
print("Output: ", attn_output)
print("Output Weight: \n", attn_output_weights)

In [ ]:
from transformers import CrossAttention

# Initialize CrossAttention
attn = CrossAttention(hidden_size=768, num_attention_heads=12, dropout=0.1, is_decoder=False)

# Calculate attention probabilities and hidden states
random_noise = torch.rand(1, 768)
random_embedding = torch.rand(1, 768)

key = attn.to_k(random_noise)
value = attn.to_v(random_noise)
query = attn.to_q(random_embedding)

attention_probs = attn.get_attention_probs(query, key, value)
print(attention_probs.shape)

hidden_states = torch.bmm(attention_probs, value)
print(hidden_states.shape)

